In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.simplefilter(action='ignore')

### LSTM Skorch example

In [33]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.pipeline import Pipeline
from skorch import NeuralNetRegressor
import torch.nn as nn
import torch
from icecream import ic

X_regr, y_regr = make_regression(
    1000,
    20,
    n_informative=10,
    random_state=0
)
X_regr = X_regr.astype(np.float32)
y_regr = y_regr.astype(np.float32)/100
y_regr = y_regr.reshape(-1,1)
ni, no, nh, nlayers = 20, 1, 10, 3

In [44]:
class LSTM(nn.Module):
    def __init__(self,ni=6,no=3,nh=10,nlayers=1):
        super(LSTM,self).__init__()
        self.ni = ni
        self.no = no
        self.nh = nh
        self.nlayers = nlayers
        self.lstms = nn.ModuleList(
            [nn.LSTMCell(self.ni,self.nh)]+[nn.LSTMCell(self.nh,self.nh) for i in range(nlayers-1)]
        )
        self.out = nn.Linear(self.nh,self.no)
        self.do = nn.Dropout(p=0.2)
        self.activation_function = nn.Tanh()
        self.dtype = torch.float
    def forward(self,x,h0=None,train=False):
        hs = x # initiate hidden state
        if h0 is None:
            h = torch.zeros(hs.shape[0],self.nh,device=x.device)
            c = torch.zeros(hs.shape[0],self.nh,device=x.device)
        else:
            (h,c) = h0
        # LSTM cells
        for i in range(self.nlayers):
            h, c = self.lstms[i](hs, (h,c))
            if train:
                hs = self.do(h)
            else:
                hs = h
        y = self.out(hs)
        return y, (h,c)

class ContextlessMSE(nn.MSELoss):
    def forward(self,y_pred,y_true):
        y, (h,c) = y_pred # extract prediction and context information
        return super().forward(y,y_true)

lstm_regressor = NeuralNetRegressor(
    module=LSTM, 
    module__ni=ni,
    module__no=no,
    module__nh=nh,
    module__nlayers=nlayers,
    max_epochs=20,
    lr=0.1,
    criterion=ContextlessMSE
)

In [45]:
lstm_regressor_pipeline = Pipeline([('lstm',lstm_regressor)])
lstm_regressor_pipeline.fit(X_regr,y_regr)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        4.7202        4.0712  0.0222
      2        4.6817        4.0438  0.0283
      3        4.6543        4.0213  0.0255
      4        4.6274        3.9967  0.0322
      5        4.5950        3.9648  0.0206
      6        4.5500        3.9174  0.0286
      7        4.4792        3.8385  0.0384
      8        4.3555        3.6909  0.0322
      9        4.1121        3.3738  0.0350
     10        3.5666        2.6097  0.0240
     11        2.3671        1.2702  0.0303
     12        1.0404        0.6253  0.0251
     13        0.6314        0.5236  0.0295
     14        0.5443        0.4820  0.0309
     15        0.4885        0.4523  0.0461
     16        0.4408        0.4340  0.0302
     17        0.4015        0.4155  0.0285
     18        0.3717        0.3600  0.0285
     19        0.3291        0.3247  0.0271
     20        0.3039        0.2730  0.0320


Pipeline(steps=[('lstm',
                 <class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=LSTM(
    (lstms): ModuleList(
      (0): LSTMCell(20, 10)
      (1): LSTMCell(10, 10)
      (2): LSTMCell(10, 10)
    )
    (out): Linear(in_features=10, out_features=1, bias=True)
    (do): Dropout(p=0.2, inplace=False)
    (activation_function): Tanh()
  ),
))])

In [52]:
# lstm_regressor_pipeline.named_steps['lstm'].get_params()
y_pred = lstm_regressor_pipeline.predict(X_regr[:5])
ic(y_pred);

ic| y_pred: array([[ 0.44321513],
                   [-1.9767034 ],
                   [-0.13864079],
                   [-0.6392553 ],
                   [-0.70358443]], dtype=float32)
